In [1]:
import numpy as np
import torch
import torch.nn as nn
from transformers import (
    CLIPModel, CLIPProcessor, GPT2LMHeadModel, GPT2Tokenizer,
    AutoModelForCausalLM, AutoTokenizer
)

/Users/aashrith/VirtualEnvs/env-dev/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/aashrith/VirtualEnvs/env-dev/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
tokenizer = AutoTokenizer.from_pretrained("gpt2")

In [3]:
tokens = tokenizer('<|endoftext|>', return_tensors='pt')

print(tokens)


{'input_ids': tensor([[50256]]), 'attention_mask': tensor([[1]])}


In [4]:
bos_token_id = tokens['input_ids'][0][0]

bos_token_id

tensor(50256)

In [5]:
model = AutoModelForCausalLM.from_pretrained("gpt2")

In [6]:
gpt2_token_emb_layer = model.transformer.wte

In [7]:
# start_token_embedding = embedding_layer(torch.IntTensor([1]))
bos_emb = gpt2_token_emb_layer(bos_token_id)


In [8]:
bos_emb.shape

torch.Size([768])

In [ ]:
import pickle

p = pickle.load(open('dataset.pkl', 'rb'))

In [55]:
p[0][1].shape

(768,)

In [3]:
from todo import CaptioningModel

In [4]:
cap_model = CaptioningModel(
    clip_model='openai/clip-vit-base-patch32',
    text_model='gpt2',
    ep_len=4,
    num_layers=1,
    n_heads=1,
    forward_expansion=4,
    max_len = 40,
    dropout = 0.1,
    device='cpu'
)

/Users/aashrith/VirtualEnvs/env-dev/lib/python3.9/site-packages/torch/nn/modules/transformer.py:307: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.num_heads is odd
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [6]:
img_emb = np.stack([t[1] for t in p[:5]])
img_emb = torch.Tensor(img_emb)

print("img_emb shape: ", img_emb.shape)

cap_model(img_emb)

img_emb shape:  torch.Size([5, 768])
after transformer:  torch.Size([5, 768])
after linear:  torch.Size([5, 3072])
shape of img_mapped:  torch.Size([5, 4, 768])
shape of bos_emb:  torch.Size([768])


RuntimeError: Tensors must have same number of dimensions: got 3 and 1

In [17]:
24576/768

32.0

In [16]:
img_emb.shape

torch.Size([768, 1])

In [19]:
x = torch.rand(2, 3)

x

tensor([[0.8493, 0.7989, 0.7470],
        [0.9589, 0.4243, 0.3711]])

In [20]:
z = x.expand(2, 2, 3)

z

tensor([[[0.8493, 0.7989, 0.7470],
         [0.9589, 0.4243, 0.3711]],

        [[0.8493, 0.7989, 0.7470],
         [0.9589, 0.4243, 0.3711]]])

In [21]:
x[0][0] = 0

x

tensor([[0.0000, 0.7989, 0.7470],
        [0.9589, 0.4243, 0.3711]])

In [22]:
z

tensor([[[0.0000, 0.7989, 0.7470],
         [0.9589, 0.4243, 0.3711]],

        [[0.0000, 0.7989, 0.7470],
         [0.9589, 0.4243, 0.3711]]])

In [28]:
tensor_3d = torch.rand(2, 3, 4)  # A random 3D tensor with shape (2, 3, 4)
tensor_1d = torch.rand(5)        # A random 1D tensor with shape (4)

In [29]:
tensor_1d

tensor([0.2767, 0.0822, 0.4346, 0.9163, 0.3621])

In [51]:
zz = tensor_1d.expand(*list(tensor_3d.shape[:-1]), tensor_1d.shape[0])

[2, 3]

In [33]:
tensor_3d.shape, zz.shape

(torch.Size([2, 3, 4]), torch.Size([2, 3, 5]))

In [34]:
c = torch.cat([tensor_3d, zz], dim=-1)

In [36]:
tensor_3d

tensor([[[0.3244, 0.6570, 0.8533, 0.0435],
         [0.4744, 0.6433, 0.3237, 0.6757],
         [0.8063, 0.6864, 0.6923, 0.6691]],

        [[0.0409, 0.4316, 0.6962, 0.1687],
         [0.8219, 0.9480, 0.7275, 0.4358],
         [0.1598, 0.6141, 0.2118, 0.6945]]])

In [37]:
zz

tensor([[[0.2767, 0.0822, 0.4346, 0.9163, 0.3621],
         [0.2767, 0.0822, 0.4346, 0.9163, 0.3621],
         [0.2767, 0.0822, 0.4346, 0.9163, 0.3621]],

        [[0.2767, 0.0822, 0.4346, 0.9163, 0.3621],
         [0.2767, 0.0822, 0.4346, 0.9163, 0.3621],
         [0.2767, 0.0822, 0.4346, 0.9163, 0.3621]]])

In [38]:
c

tensor([[[0.3244, 0.6570, 0.8533, 0.0435, 0.2767, 0.0822, 0.4346, 0.9163,
          0.3621],
         [0.4744, 0.6433, 0.3237, 0.6757, 0.2767, 0.0822, 0.4346, 0.9163,
          0.3621],
         [0.8063, 0.6864, 0.6923, 0.6691, 0.2767, 0.0822, 0.4346, 0.9163,
          0.3621]],

        [[0.0409, 0.4316, 0.6962, 0.1687, 0.2767, 0.0822, 0.4346, 0.9163,
          0.3621],
         [0.8219, 0.9480, 0.7275, 0.4358, 0.2767, 0.0822, 0.4346, 0.9163,
          0.3621],
         [0.1598, 0.6141, 0.2118, 0.6945, 0.2767, 0.0822, 0.4346, 0.9163,
          0.3621]]])

In [40]:
torch.cat([tensor_3d, tensor_1d.unsqueeze(0).unsqueeze(0)])

RuntimeError: Sizes of tensors must match except in dimension 0. Expected size 3 but got size 1 for tensor number 1 in the list.